# EZ Model

Notebook to drive model discovery, data load, asking questions.

You discover model by chatting with an LLM model, specifically Anthropic Claude V2 via Amazon Bedrock. You visualize the model using PlantUML, a diagram-as-code tool. 

Once you're happy with the model, you create test data, again with help from the LLM.

You then load that data into Amazon Neptune database.

Then you ask it questions. For this we use Langchain's Neptune OpenCypher integration with LLM. The question/answer flow is the following:
- During setup, the Langchain Neptune OpenCypher component introspects the Neptune database and extracts a graph schema.
- The end user asks as a natural language question.
- The LLM writes an OpenCypher query representing the intent of the question. It uses the graph schema extracted above.
- The Langchain Neptune OpenCypher component executes the query and returns back the result, represented as a JSON object.
- The LLM answers the end user's question in natural language based on the query result.

## Setup
### Pre-Req
This notebook is meant to run in an Amazon Neptune notebook. We recommend using the Quickstart setup to create a Neptune cluster and notebook. See https://docs.aws.amazon.com/neptune/latest/userguide/get-started-cfn-create.html. 

Your Neptune cluster must run engine version 1.2.1.O.R6 or higher. 

Python 3.10 or higher is required in the notebook instance. If you are running an older notebook instance, we recommend creating a new instance to use this notebook.

This notebook uses Amazon Bedrock. You must setup Neptune and the notebook in a region supported by Bedrock. See https://docs.aws.amazon.com/bedrock/latest/userguide/what-is-bedrock.html for supported regions. We tested this in us-east-1.

### IAM policy for Amazon Bedrock
Modify the IAM role for your notebook instance with a policy allowing access to Bedrock. You can find that role in the SageMaker console under Notebooks. Select your notebook instance to see its details, including IAM role. Edit the role by adding the following. 

        {
            "Action": [
                "bedrock:ListFoundationModels",
                "bedrock:InvokeModel"
            ],
            "Resource": "*",
            "Effect": "Allow"
        }
### Install Python dependencies
Run the following cells to install required libraries.

In [ ]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

In [ ]:
%pip install langchain==0.0.309

In [ ]:
%pip install iplantuml

### Restart the kernel
To use the libraries installed above, restart the notebook kernel. In Jupyter, choose menu Kernel | Restart and Clear Output.

## Create conversational client allowing you to ask questions to LLM
Chat with Bedrock Claude V2 large language model (LLM) to create a graph data model for a company exclusion list. Then use PlantUML to visualize the grah model. First setup the conversational interface.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def extract_codeblocks(res):
    curr_block = None
    blocks = []
    toks = res.split("```")
    for t in toks:
        if curr_block is None:
            curr_block = {'text': "", 'type': ""}
        else:
            first_nl = t.find("\n")
            curr_block['type'] = t[0:first_nl]
            curr_block['text'] = t[first_nl:].strip()
            blocks.append(curr_block)
            curr_block = None
    return blocks

In [ ]:
import boto3
from langchain.chains import ConversationChain
from langchain.chat_models import BedrockChat
from langchain.llms import Bedrock
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate, 
    SystemMessagePromptTemplate, 
    MessagesPlaceholder, 
    HumanMessagePromptTemplate, 
    PromptTemplate
)

'''
Useful material on chat with claude
https://github.com/langchain-ai/langchain/issues/11220
https://github.com/aws-samples/amazon-bedrock-workshop/blob/main/04_Chatbot/00_Chatbot_Claude.ipynb
'''


bedrock_client = boto3.client('bedrock-runtime')
llm = BedrockChat(
    model_id = "anthropic.claude-v2",
    client = bedrock_client
)

# turn verbose to true to see the full logs and documents
conversation= ConversationChain(
    llm=llm, verbose=False, memory=ConversationBufferMemory() #memory_chain
)

# langchain prompts do not always work with all the models. This prompt is tuned for Claude
claude_prompt = PromptTemplate.from_template("""

Human: The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context. If the AI does not know
the answer to a question, it truthfully says it does not know.

Current conversation:
<conversation_history>
{history}
</conversation_history>

Here is the human's next reply:
<human_reply>
{input}
</human_reply>

Assistant:
""")

conversation.prompt = claude_prompt

print(conversation.predict(input="Hi there!"))

###  Show a knowledge graph model for a company exclusion list
<details><summary>Click to view/hide sample answer</summary>
<p>
A knowledge graph representing a company exclusion list could potentially model entities like companies, sectors, regions, and reasons for exclusion. It might link together companies with attributes like industry, location, business practices, and controversy level. The graph could illustrate why certain companies are excluded by connecting them to concepts like human rights violations, environmental damage, or corruption. Relationships like "company X is excluded for reason Y" could be modeled. Overall the knowledge graph could provide a structured, visual representation of the exclusions, their justifications, and the relationships between entities. But without access to actual exclusion list data, I can only speculate about what such a knowledge graph might look like. Please let me know if you would like me to elaborate on any part of this general response!

</p>
</details>

In [ ]:
print(conversation.run('''Show a knowledge graph model for a company exclusion list.'''))



### Show as PlantUML class diagram
<details><summary>Click to view/hide sample answer</summary>
<p>
Here is the company exclusion list knowledge graph as a PlantUML class diagram presented in a code block:

```plantuml
@startuml

class Company {
  - name
  - sector
  - region
  - controversy_level 
}

class ExclusionReason {
  - name
  - description  
}

class ExclusionList {
  - name
  - owner 
}

Company "1" -- "0..*" ExclusionReason : is_excluded_for > 
ExclusionList "1" -- "0..*" Company : contains >

class Sector {
  - name  
}

Company "1" -- "1" Sector : belongs_to >

class Region {
  - name
}

Company "1" -- "1" Region : located_in >


class Controversy {
  - category
  - description  
}

Company "1" -- "0..*" Controversy : involved_in >

@enduml
```

Presenting the PlantUML diagram as code allows it to be easily shared and modified as plain text. Let me know if you would like me to explain or build on this
 </p>
</details>

In [ ]:
res = conversation.run('''Human: Show as a PlantUML class diagram. Present as code block.''')
uml = extract_codeblocks(res)
print(res)

### Show PlantUML diagram
<details><summary>Click to view/hide sample image</summary>
<p>
    <img src="https://raw.githubusercontent.com/aws-samples/amazon-neptune-ontology-example-blog/main/plantuml/plantuml1.png"/>   
</p>
</details>

In [ ]:
import iplantuml

In [ ]:
print(str(len(uml)))
ipython = get_ipython()
ipython.run_cell_magic("plantuml", "", uml[0]['text'])

### Include how companies related to each other
<details><summary>Click to view/hide sample answer</summary>
<p>
Here is an updated PlantUML class diagram that models how companies can be related to each other, presented as a code block:

```plantuml
@startuml

class Company {
  - name  
  - sector
  - region
  - controversy_level
}

class ExclusionReason {
  - name 
  - description   
}

class ExclusionList {
  - name
  - owner  
}

Company "1" -- "0..*" ExclusionReason : is_excluded_for >
ExclusionList "1" -- "0..*" Company : contains > 

class Sector {
  - name   
}

Company "1" -- "1" Sector : belongs_to >

class Region {
  - name  
}

Company "1" -- "1" Region : located_in >


class Controversy {
  - category
  - description   
}

Company "1" -- "0..*" Controversy : involved_in >


Company "0..*" -- "0..*" Company : related_to >

@enduml
```

The key addition is the "related_to" relationship between Company
</p>
</details>

In [ ]:
res = conversation.run('''Include how companies related to each other. Present as PlantUML codeblock''')
uml = extract_codeblocks(res)
print(res)

### Show change in PlantUML
<details><summary>Click to view/hide sample image</summary>
<p>
    <img src="https://raw.githubusercontent.com/aws-samples/amazon-neptune-ontology-example-blog/main/plantuml/plantuml2.png"/>   
</p>
</details>

In [ ]:
ipython = get_ipython()
ipython.run_cell_magic("plantuml", "", uml[0]['text'])

### Let's tweak a bit

In [ ]:
good_model_content = """

left to right direction
class Company {
    +Name: String
    +Industry: String
    +Location: String
}

class ExclusionCriteria {
    +Type: String
    +Description: String
    +SeverityLevel: String
}

class Exclusion {
    +ExclusionStartDate: Date
    +ExclusionEndDate: Date
    +ExclusionType: String
}

Company "0" --> "*" Exclusion : has exclusion 
Company "0" --> "0..1" Exclusion : has current exclusion 
Exclusion "1" --> "1" ExclusionCriteria : has criteria

Company "1" --> "*" Company: related to
note on link : reltype: String, established date: Date
"""

good_model= "@startuml\n" + good_model_content + "\n@enduml\n"



### Tweaked PlantUML
<details><summary>Click to view/hide image</summary>
<p>
    <img src="https://raw.githubusercontent.com/aws-samples/amazon-neptune-ontology-example-blog/main/plantuml/plantuml3.png"/>   
</p>
</details>

In [ ]:
ipython = get_ipython()
ipython.run_cell_magic("plantuml", "", good_model)

## Get some test data for our model

<details><summary>Click to view/hide sample answers</summary>
<p>
    
Here is what sample response for OpenCypher looks like:

```
// Create sample companies
CREATE 
  (acme:Company {name: "Acme Co", industry: "Manufacturing", location: "New York"}),
  (globex:Company {name: "Globex Corp", industry: "Technology", location: "San Francisco"}),
  (contoso:Company {name: "Contoso Ltd", industry: "Retail", location: "London"})
  
// Create sample exclusion criteria  
CREATE
  (criteria1:ExclusionCriteria {type: "Legal", description: "Ongoing investigation", severity: "High"}),
  (criteria2:ExclusionCriteria {type: "Regulatory", description: "Sanctions violations", severity: "Critical"})
  
// Create sample exclusions
CREATE 
  (acme)-[:HAS_EXCLUSION]->(exclusion1:Exclusion {startDate: date('2019-01-01'), endDate: date('2020-12-31'), type: "Legal"}),
  (globex)-[:HAS_CURRENT_EXCLUSION]->(exclusion2:Exclusion {startDate: date('2021-01-01'), endDate: null, type
```
    
And sample for RDF:

```
@prefix : <http://example.com/>
@prefix xsd: <http://www.w3.org/2001/XMLSchema#>

# Companies
:acme a :Company; 
    :name "Acme Co"; 
    :industry "Manufacturing";
    :location "New York".

:globex a :Company;
   :name "Globex Corp";
   :industry "Technology"; 
   :location "San Francisco".
   
:contoso a :Company;
   :name "Contoso Ltd";
   :industry "Retail";
   :location "London".
   
# Exclusion criteria
:criteria1 a :ExclusionCriteria;
  :type "Legal";
  :description "Ongoing investigation";
  :severityLevel "High".  

:criteria2 a :ExclusionCriteria;
  :type "Regulatory";
  :description "Sanctions violations";
  :severityLevel "Critical".
  
# Exclusions  
:exclusion1 a :Exclusion;
  :exclusionStartDate "2019-01-01"^^xsd:date;
  :exclusionEndDate "2020
```
    
</p>
</details>

In [ ]:
res = conversation.run('''Human: Generate OpenCypher queries to create sample data based on ''' + good_model_content)
print(res)

In [ ]:
res = conversation.run('''Human: Generate RDF sample data based on ''' + good_model_content)
print(res)

## Load the data from OC into Neptune
We'll break this into two sets of OpenCypher inserts: nodes and edges. 

In [ ]:
%%oc

CREATE (:Company {`~id`: 'compA', Name: 'Company A', Industry: 'Tech', Location: 'California', Tag: 'demo'})
CREATE (:Company {`~id`: 'compB', Name: 'Company B', Industry: 'Finance', Location: 'New York', Tag: 'demo'})
CREATE (:Company {`~id`: 'compC', Name: 'Company C', Industry: 'Retail', Location: 'Texas', Tag: 'demo'})
CREATE (:Company {`~id`: 'compD', Name: 'Company D', Industry: 'Retail', Location: 'Texas', Tag: 'demo'})
CREATE (:Company {`~id`: 'compE', Name: 'Company E', Industry: 'Retail', Location: 'Texas', Tag: 'demo'})

CREATE (:ExclusionCriteria {`~id`: 'xfin1', Type: 'Financial', Description: 'Non-payment of invoices', SeverityLevel: 'High', Tag: 'demo'})
CREATE (:ExclusionCriteria {`~id`: 'xfin2', Type: 'Financial', Description: 'Rating devaluation', SeverityLevel: 'Medium', Tag: 'demo'})
CREATE (:ExclusionCriteria {`~id`: 'xleg', Type: 'Legal', Description: 'Violation of labor laws', SeverityLevel: 'Medium', Tag: 'demo'})
CREATE (:ExclusionCriteria {`~id`: 'xeth',Type: 'Ethical', Description: 'Unethical business practices', SeverityLevel: 'Low', Tag: 'demo'})

CREATE (:Exclusion {`~id`: 'xa2', ExclusionStartDate: datetime('2022-01-01T00:00:01'), ExclusionType: 'Financial', Tag: 'demo'})
CREATE (:Exclusion {`~id`: 'xa1', ExclusionStartDate: datetime('2020-01-01T00:00:01'), ExclusionEndDate: datetime('2020-06-11T00:00:01'), ExclusionType: 'Financial', Tag: 'demo'})
CREATE (:Exclusion {`~id`: 'xb1', ExclusionStartDate: datetime('1985-01-01T00:00:01'), ExclusionType: 'Legal', Tag: 'demo'})
CREATE (:Exclusion {`~id`: 'xc1', ExclusionStartDate: datetime('1985-01-01T00:00:01'), ExclusionStartDate: datetime('1989-01-01T00:00:01'), ExclusionType: 'Legal', Tag: 'demo'})
                                 

In [ ]:
%%oc

MATCH (companyA:Company {`~id`: 'compA'})                  
MATCH (companyB:Company {`~id`: 'compB'})       
MATCH (companyC:Company {`~id`: 'compC'})       
MATCH (companyD:Company {`~id`: 'compD'})       
MATCH (xa2:Exclusion {`~id`: 'xa2'})
MATCH (xa1:Exclusion {`~id`: 'xa1'})
MATCH (xb1:Exclusion {`~id`: 'xb1'})
MATCH (xc1:Exclusion {`~id`: 'xc1'})
CREATE (companyA)-[r1:relatedTo {reltype: 'Partner', establishedDate: datetime('2020-01-01T00:00:01')}]->(companyB)
CREATE (companyA)-[r2:relatedTo {reltype: 'Supplier', establishedDate: datetime('2021-01-01T00:00:01')}]->(companyC)
CREATE (companyC)-[r3:relatedTo {reltype: 'Supplier', establishedDate: datetime('2021-01-01T00:00:01')}]->(companyD)
CREATE (companyA)-[x1:hasCurrentExclusion]->(xa2)
CREATE (companyA)-[x2:hasExclusion]->(xa2)
CREATE (companyA)-[x3:hasExclusion]->(xa1)
CREATE (companyB)-[x4:hasCurrentExclusion]->(xb1)
CREATE (companyB)-[x5:hasExclusion]->(xb1)
CREATE (companyC)-[x6:hasExclusion]->(xc1)



In [ ]:
%%oc

MATCH (xfin1:ExclusionCriteria {`~id`: 'xfin1'})
MATCH (xfin2:ExclusionCriteria {`~id`: 'xfin2'})
MATCH (xleg:ExclusionCriteria {`~id`: 'xleg'})
MATCH (xa2:Exclusion {`~id`: 'xa2'})
MATCH (xa1:Exclusion {`~id`: 'xa1'})
MATCH (xb1:Exclusion {`~id`: 'xb1'})
MATCH (xc1:Exclusion {`~id`: 'xc1'})
CREATE (xa2)-[c1:hasCriteria]->(xfin1)
CREATE (xa1)-[c2:hasCriteria]->(xfin2)
CREATE (xb1)-[c3:hasCriteria]->(xfin1)
CREATE (xc1)-[c4:hasCriteria]->(xleg)


## Ask questions
Now let's use the LLM to test-drive the graph model. We use OpenCypher to query the mock data that we ingested above. We can express queries as natural language prompts. The LLM will formulate an OpenCypher query for the prompt. 

We begin by setting up the plumbing to enable this. We first create a NeptuneGraph object that introspects the schema of the Neptune cluster. Then we create a NeptuneOpenCypherQAChain object that uses the LLM to formulate the queries using the schema from Neptune. NeptuneGraph also runs the query on the LLM's behalf. The LLM interprets the query result and reports back with a natural language response.

In [ ]:
import os

# Grab Neptune cluster host/port from notebook instance environment variables
GRAPH_NOTEBOOK_HOST= os.popen("source ~/.bashrc ; echo $GRAPH_NOTEBOOK_HOST").read().split("\n")[0]
GRAPH_NOTEBOOK_PORT= os.popen("source ~/.bashrc ; echo $GRAPH_NOTEBOOK_PORT").read().split("\n")[0]
[GRAPH_NOTEBOOK_HOST, GRAPH_NOTEBOOK_PORT]

In [ ]:
import boto3
from langchain.chains import NeptuneOpenCypherQAChain
from langchain.graphs import NeptuneGraph

graph = NeptuneGraph(host=GRAPH_NOTEBOOK_HOST, port=GRAPH_NOTEBOOK_PORT)
chain = NeptuneOpenCypherQAChain.from_llm(
    llm = llm, graph = graph, verbose = True, top_K = 10, return_intermediate_steps=True, return_direct=False
)
graph.schema

### Q1 - Which companies have financial exclusions

<details><summary>Click to view/hide sample answer</summary>
<p>


> Entering new NeptuneOpenCypherQAChain chain...
Generated Cypher:
 MATCH (c:Company)-[:hasExclusion]->(e:Exclusion) 
WHERE e.ExclusionType = "Financial"
RETURN c.Name

Full Context:
{'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked', 'content-type': 'application/json;charset=UTF-8'}, 'RetryAttempts': 0}, 'results': [{'c.Name': 'Company A'}, {'c.Name': 'Company A'}]}

> Finished chain.

" Unfortunately I do not have enough information to determine which companies have exclusions of type Financial that are not current. The provided information only includes company names, without any details about exclusions. Since there are no exclusions mentioned, I don't know the answer to your question about companies with non-current Financial exclusions."
    
</p>
</details>

In [ ]:
chain.run('''Which companies have exclusions of type Financial. The exclusions need not be current.''')

### Q2 - Which companies have current exclusions

<details><summary>Click to view/hide sample answer</summary>
<p>


> Entering new NeptuneOpenCypherQAChain chain...
Generated Cypher:
 MATCH (c:Company)-[:hasCurrentExclusion]->(e:Exclusion)
RETURN c

Full Context:
{'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked', 'content-type': 'application/json;charset=UTF-8'}, 'RetryAttempts': 0}, 'results': [{'c': {'~id': 'compA', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'California', 'Industry': 'Tech', 'Name': 'Company A'}}}, {'c': {'~id': 'compB', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'New York', 'Industry': 'Finance', 'Name': 'Company B'}}}]}

> Finished chain.

" Unfortunately, the provided information does not contain any details about current exclusions for the companies. The information only includes basic details like name, location, industry and tag for Company A and Company B. Since there is no data on exclusions, I don't have enough information to determine which companies may have current exclusions."    
</p>
</details>

In [ ]:
chain.run('''Which companies have current exclusions. Hint: no need to check ExclusionEndDate''')

### Q3 - Which companies do not have a current exclusion
<details><summary>Click to view/hide sample answer</summary>
<p>
    


> Entering new NeptuneOpenCypherQAChain chain...
Generated Cypher:
 MATCH (c:Company) 
WHERE NOT (c)-[:hasCurrentExclusion]->()
RETURN c

Full Context:
{'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked', 'content-type': 'application/json;charset=UTF-8'}, 'RetryAttempts': 0}, 'results': [{'c': {'~id': 'compC', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'Texas', 'Industry': 'Retail', 'Name': 'Company C'}}}, {'c': {'~id': 'compD', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'Texas', 'Industry': 'Retail', 'Name': 'Company D'}}}, {'c': {'~id': 'compE', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'Texas', 'Industry': 'Retail', 'Name': 'Company E'}}}]}

> Finished chain.

' Based on the provided information, there are 3 companies listed: Company C, Company D, and Company E. None of them have any information indicating they currently have an exclusion. So it seems that none of the companies listed have a current exclusion.'
</p>
</details>

In [ ]:
chain.run('''Which companies do not have a current exclusion''')

### Q4 - Which companies located in Texas have an exclusion or are directly related to a company with an exclusion
<details><summary>Click to view/hide sample answer</summary>
<p>


> Entering new NeptuneOpenCypherQAChain chain...
Generated Cypher:
 MATCH (c:Company)-[:relatedTo|hasCurrentExclusion|hasExclusion*]-(e:Exclusion)
WHERE c.Location = "Texas"
RETURN c

Full Context:
{'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked', 'content-type': 'application/json;charset=UTF-8'}, 'RetryAttempts': 0}, 'results': [{'c': {'~id': 'compC', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'Texas', 'Industry': 'Retail', 'Name': 'Company C'}}}, {'c': {'~id': 'compC', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'Texas', 'Industry': 'Retail', 'Name': 'Company C'}}}, {'c': {'~id': 'compC', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'Texas', 'Industry': 'Retail', 'Name': 'Company C'}}}, {'c': {'~id': 'compC', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'Texas', 'Industry': 'Retail', 'Name': 'Company C'}}}, {'c': {'~id': 'compC', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'Texas', 'Industry': 'Retail', 'Name': 'Company C'}}}, {'c': {'~id': 'compC', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'Texas', 'Industry': 'Retail', 'Name': 'Company C'}}}, {'c': {'~id': 'compD', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'Texas', 'Industry': 'Retail', 'Name': 'Company D'}}}, {'c': {'~id': 'compD', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'Texas', 'Industry': 'Retail', 'Name': 'Company D'}}}, {'c': {'~id': 'compD', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'Texas', 'Industry': 'Retail', 'Name': 'Company D'}}}, {'c': {'~id': 'compD', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'Texas', 'Industry': 'Retail', 'Name': 'Company D'}}}, {'c': {'~id': 'compD', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'Texas', 'Industry': 'Retail', 'Name': 'Company D'}}}, {'c': {'~id': 'compD', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'Texas', 'Industry': 'Retail', 'Name': 'Company D'}}}, {'c': {'~id': 'compC', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'Texas', 'Industry': 'Retail', 'Name': 'Company C'}}}, {'c': {'~id': 'compC', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'Texas', 'Industry': 'Retail', 'Name': 'Company C'}}}, {'c': {'~id': 'compC', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'Texas', 'Industry': 'Retail', 'Name': 'Company C'}}}, {'c': {'~id': 'compC', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'Texas', 'Industry': 'Retail', 'Name': 'Company C'}}}, {'c': {'~id': 'compC', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'Texas', 'Industry': 'Retail', 'Name': 'Company C'}}}, {'c': {'~id': 'compC', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'Texas', 'Industry': 'Retail', 'Name': 'Company C'}}}, {'c': {'~id': 'compD', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'Texas', 'Industry': 'Retail', 'Name': 'Company D'}}}, {'c': {'~id': 'compD', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'Texas', 'Industry': 'Retail', 'Name': 'Company D'}}}, {'c': {'~id': 'compD', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'Texas', 'Industry': 'Retail', 'Name': 'Company D'}}}, {'c': {'~id': 'compD', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'Texas', 'Industry': 'Retail', 'Name': 'Company D'}}}, {'c': {'~id': 'compD', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'Texas', 'Industry': 'Retail', 'Name': 'Company D'}}}, {'c': {'~id': 'compD', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'Texas', 'Industry': 'Retail', 'Name': 'Company D'}}}]}

> Finished chain.

' Based on the provided information, the companies with a location of Texas that have an exclusion or are directly related to a company with an exclusion are:\n\nCompany C and Company D. \n\nThe information shows multiple nodes for Company C and Company D, both with a location of Texas. It does not specify any exclusions, but I do not have enough information to determine if they have exclusions or are related to companies with exclusions. The provided information does not indicate any other relevant companies.'
</p>
</details>

In [ ]:
chain.run('''Which companies with location of Texas have an exclusion or are directly related to a company with an exclusion''')

### Q5 - Which companies are related directly to Company A
<details><summary>Click to view/hide sample answer</summary>
<p>
    


> Entering new NeptuneOpenCypherQAChain chain...
Generated Cypher:
 MATCH (c1:Company {Name:"Company A"})-[:relatedTo]->(c2:Company)
RETURN c2

Full Context:
{'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked', 'content-type': 'application/json;charset=UTF-8'}, 'RetryAttempts': 0}, 'results': [{'c2': {'~id': 'compC', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'Texas', 'Industry': 'Retail', 'Name': 'Company C'}}}, {'c2': {'~id': 'compB', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'New York', 'Industry': 'Finance', 'Name': 'Company B'}}}]}

> Finished chain.

" Unfortunately, the provided information does not contain any information about Company A or its direct relationships with other companies. Without that context, I don't have enough information to determine which companies are related directly to Company A."
</p>
</details>

In [ ]:
chain.run('''Which companies are related directly to Company A''')

### Q6 - Which companies are related directly or indirectly to Company A
<details><summary>Click to view/hide sample answer</summary>
<p>
    
> Entering new NeptuneOpenCypherQAChain chain...
Generated Cypher:
 MATCH (a:Company {Name:"Company A"})-[:relatedTo*1..]-(company) 
RETURN company

Full Context:
{'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked', 'content-type': 'application/json;charset=UTF-8'}, 'RetryAttempts': 0}, 'results': [{'company': {'~id': 'compD', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'Texas', 'Industry': 'Retail', 'Name': 'Company D'}}}, {'company': {'~id': 'compB', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'New York', 'Industry': 'Finance', 'Name': 'Company B'}}}, {'company': {'~id': 'compC', '~entityType': 'node', '~labels': ['Company'], '~properties': {'Tag': 'demo', 'Location': 'Texas', 'Industry': 'Retail', 'Name': 'Company C'}}}]}

> Finished chain.

" Based on the provided information, there are no companies related directly or indirectly to Company A. The information only contains details about Company B located in New York in the finance industry, Company C located in Texas in the retail industry, and Company D located in Texas in the retail industry. Since Company A is not mentioned, I don't have enough information to identify any companies related to it."

</p>
</details>

In [ ]:
chain.run('''Which companies are related directly or indirectly to Company A''')